In [ ]:
! pip install -U -r environment/requirenments.txt
! pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
! python -m spacy download en_core_web_lg
! python -m spacy download en_core_web_sm

In [3]:
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
# from autocorrect import Speller
import json
import inspect
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
# from sentence_transformers import SentenceTransformer, util
import torch
import tokenize
from tokenize import TokenError
import io
tqdm.pandas()

import nltk
import re
import string

import numpy as np
import pandas as pd
from scipy import sparse
import sys
import textwrap
import wandb
# import spacy
# import contextualSpellCheck
from bisect import bisect
from sklearn.preprocessing import FunctionTransformer
# ner = spacy.load("en_core_web_lg")
# check_spelling = spacy.load("en_core_web_sm")

from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('data/')
sys.path.append(str('AI4Code'))
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

def read_train_data(data_dir, NUM_TRAIN = 10000, OFFSET=0):
    def read_notebook(path):
        return (
            pd.read_json(
                path,
                dtype={'cell_type': 'category', 'source': 'str'})
            .assign(id=path.stem)  # final path component
            .rename_axis('cell_id')
        )

    paths_train = list((data_dir / 'train').glob('*.json'))[OFFSET:NUM_TRAIN]
    notebooks_train = [
      read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
    ]
    df = (
      pd.concat(notebooks_train)
      .set_index('id', append=True)
      .swaplevel()
      .sort_index(level='id', sort_remaining=False)
    )
    return df

def get_df_orders_and_ranks(df, data_dir):
    # train orders
    df_orders = pd.read_csv(
      data_dir / 'train_orders.csv',
      index_col='id',
      squeeze=True,
    ).str.split()  # cell_ids str -> list


    df_orders_ = df_orders.to_frame().join(
      # reset only one index out of many -> "cell_id"; make a list out of cells in train data
      df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
      how='right',
    )

    ranks = {}
    for id_, cell_order, cell_id in df_orders_.itertuples():
        ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}

    df_ranks = (
      pd.DataFrame
      .from_dict(ranks, orient='index')
      .rename_axis('id')
      .apply(pd.Series.explode)
      .set_index('cell_id', append=True)
    )
    # now we have
    # id cell_id rank
    return df_orders, df_ranks


def get_ancestors(data_dir, ids):
    # Split, keeping notebooks with a common origin (ancestor_id) together
    df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')
    return df_ancestors.loc[ids, 'ancestor_id']

# Getting the data

In [ ]:
# setting the parameters
NVALID = 0.1  # size of validation set

In [112]:
# example pipeline work
df = read_train_data(data_dir, NUM_TRAIN=1000)
df_orders, df_ranks = get_df_orders_and_ranks(df, data_dir)

KeyboardInterrupt: 

In [ ]:
#splitting on markdown and code
markdowns = df[df['cell_type'] == 'markdown']
codes = df[df['cell_type'] == 'code']
df = pd.concat([codes, markdowns])

# Splitting

In [ ]:
splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)
ids = df.index.unique('id')
ancestors = get_ancestors(data_dir, ids)  # find ancestor by id if it exists
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors)) 
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

In [ ]:
sparse_matrix = sparse.load_npz('data/df_tfisd_15k.npz').A[:len(df)]

In [54]:
# for getting original df
df_train = df[df.index.get_level_values('id').isin(ids_train)]
df_valid = df[df.index.get_level_values('id').isin(ids_valid)]

In [55]:
X_train = sparse_matrix[
    (df.index.get_level_values('id').isin(ids_train))
]
X_valid = sparse_matrix[
    (df.index.get_level_values('id').isin(ids_valid))
]

In [84]:
X_train = np.hstack((
    X_train,
    np.where(
        df_train['cell_type'] == 'code',
        df_train.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1))
)

In [85]:
X_valid = np.hstack((
    X_valid,
    np.where(
        df_valid['cell_type'] == 'code',
        df_valid.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1))
)

In [86]:
# get all required train results
y_train = df_ranks.loc[ids_train].to_numpy()
# Number of cells in each notebook. will later be used to help xgboost make a ranking
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy()

In [87]:
y_valid = df_orders.loc[ids_valid]

# Model

# xgb

In [93]:
from xgboost import XGBRanker

model = XGBRanker()

In [94]:
model.fit(X_train, y_train, group=groups)

XGBRanker(base_score=0.5, booster='gbtree', callbacks=None, colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, early_stopping_rounds=None,
          enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
          grow_policy='depthwise', importance_type=None,
          interaction_constraints='', learning_rate=0.300000012, max_bin=256,
          max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
          min_child_weight=1, missing=nan, monotone_constraints='()',
          n_estimators=100, n_jobs=0, num_parallel_tree=1,
          objective='rank:pairwise', predictor='auto', random_state=0,
          reg_alpha=0, ...)

## Pytorch use language code

In [27]:
from tqdm import tqdm
import sys, os
from transformers import DistilBertModel, DistilBertTokenizer
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch

In [28]:
MAX_LEN = 128

In [29]:
class MarkdownModel(nn.Module):
    def __init__(self):
        super(MarkdownModel, self).__init__()
        self.distill_bert = DistilBertModel.from_pretrained(BERT_PATH)

        self.top1 = nn.Linear(768, 64)
        self.top2 = nn.Linear(64, 1)

        self.dropout1 = torch.nn.Dropout(p=0.2)
        self.dropout2 = torch.nn.Dropout(p=0.2)
        
    def forward(self, ids, mask):
        x = self.distill_bert(ids, mask)[0][:, 0, :]
        x = self.dropout1(x)
        x0 = self.top1(x)
        x = self.dropout2(x0)
        x = self.top2(x)
        x = torch.sigmoid(x)
        return x

In [34]:
BERT_PATH = "distilbert-base-uncased"

In [ ]:
val_df_mark.iloc[0]

# Prediction

In [95]:
y_pred = pd.DataFrame({'rank': model.predict(X_valid)}, index=df_valid.index)
y_pred = (
    y_pred
    .sort_values(['id', 'rank'])  # Sort the cells in each notebook by their rank.
                                  # The cell_ids are now in the order the model predicted.
    .reset_index('cell_id')  # Convert the cell_id index into a column.
    .groupby('id')['cell_id'].apply(list)  # Group the cell_ids for each notebook into a list.
)

## Metrics 

In [62]:
def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max


# Results

In [96]:
y_dummy = df_valid.reset_index('cell_id').groupby('id')['cell_id'].apply(list)
kendall_tau(y_valid, y_dummy)

0.283291481553023

In [109]:
kendall_tau(y_valid, y_pred)

-0.03883330113965622

## Looking at the data

In [107]:
nb_id = df_valid.index.get_level_values('id').unique()[13]

df.loc[nb_id].loc[y_valid.loc[nb_id]][['source', 'cell_type']].head(15)

,source,cell_type
cell_id,,
ae6e8718,"You've built a model. But how good is it?\n\nIn this lesson, you will learn to use model validation to measure the q...",markdown
0f981241,# Data Loading Code Hidden Here\nimport pandas as pd\n\n# Load data\nmelbourne_file_path = '../input/melbourne-housi...,code
063d7406,"Once we have a model, here is how we calculate the mean absolute error:",markdown
b348f80e,from sklearn.metrics import mean_absolute_error\n\npredicted_home_prices = melbourne_model.predict(X)\nmean_absolute...,code
a39c4598,"# The Problem with ""In-Sample"" Scores\n\nThe measure we just computed can be called an ""in-sample"" score. We used a ...",markdown
bd5bab25,"from sklearn.model_selection import train_test_split\n\n# split data into training and validation data, for both fea...",code
42384b5d,"# Wow!\n\nYour mean absolute error for the in-sample data was about 500 dollars. Out-of-sample it is more than 250,...",markdown
465e607e,"# Your Turn\nBefore we look at improving this model, try **[Model Validation](https://www.kaggle.com/kernels/fork/22...",markdown


In [108]:
df.loc[nb_id].loc[y_pred.loc[nb_id]][['source', 'cell_type']].head(15)

,source,cell_type
cell_id,,
063d7406,"Once we have a model, here is how we calculate the mean absolute error:",markdown
465e607e,"# Your Turn\nBefore we look at improving this model, try **[Model Validation](https://www.kaggle.com/kernels/fork/22...",markdown
ae6e8718,"You've built a model. But how good is it?\n\nIn this lesson, you will learn to use model validation to measure the q...",markdown
a39c4598,"# The Problem with ""In-Sample"" Scores\n\nThe measure we just computed can be called an ""in-sample"" score. We used a ...",markdown
0f981241,# Data Loading Code Hidden Here\nimport pandas as pd\n\n# Load data\nmelbourne_file_path = '../input/melbourne-housi...,code
bd5bab25,"from sklearn.model_selection import train_test_split\n\n# split data into training and validation data, for both fea...",code
42384b5d,"# Wow!\n\nYour mean absolute error for the in-sample data was about 500 dollars. Out-of-sample it is more than 250,...",markdown
b348f80e,from sklearn.metrics import mean_absolute_error\n\npredicted_home_prices = melbourne_model.predict(X)\nmean_absolute...,code
